In [6]:
import numpy as np
import torch
from PetDataset import PetDataset
import torchvision.transforms as transforms
import pandas as pd
import matplotlib.pyplot as plt
from Functions import balance_df

df = pd.read_csv("petfinder-pawpularity-score/train.csv")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
P_SIZE=256

cuda:0


In [7]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df,test_size=0.2)
train_df = balance_df(train_df)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((P_SIZE, P_SIZE)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = PetDataset(data_transform=transform,data_csv=train_df)
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=4,
                                               shuffle=True,
                                               num_workers=4)

test_dataset = PetDataset(data_transform=transform,data_csv=test_df)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=4,
                                              shuffle=True,
                                              num_workers=4)


In [8]:
def out_size(h, k, p=0,dd=1,s=1):
    return int((h+2*p-dd*(k-1)-1)/s + 1)
a=out_size(64, 5)
print(a)
b=out_size(a, 2, 0, 1, 2)
print(b)
c=out_size(b, 5)
print(c)
d=out_size(c,2,0,1,2)
print(d)

60
30
26
13


In [9]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, P_SIZE=256):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.a = out_size(P_SIZE, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.b = out_size(self.a, 2, 0, 1, 2)

        self.conv2 = nn.Conv2d(16, 16, 5)
        self.c = out_size(self.b, 5)
        self.d = out_size(self.c, 2, 0, 1, 2)

        # self.conv3 = nn.Conv2d(16, 5, 5)
        # self.e = out_size(self.d, 5)
        # self.f = out_size(self.e, 2, 0, 1, 2)

        self.fc1 = nn.Linear(16 * self.d * self.d, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)
        self.fc4 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 16 * self.d * self.d)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        # x = (100*x.view(-1)).int()/100
        # x.requires_grad=True
        return x.view(-1)

In [ ]:
import torch.optim as optim
net = Net()
net.to(device)
criterion = nn.MSELoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
for epoch in range(10):

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


[1,   200] loss: 0.082
[1,   400] loss: 0.087
[1,   600] loss: 0.080
[1,   800] loss: 0.083
[1,  1000] loss: 0.087
[1,  1200] loss: 0.084
[1,  1400] loss: 0.082


In [ ]:
iter_i = 0

with torch.no_grad():
    for data in test_dataloader:
        inputs, _ = data[0].to(device), data[1].to(device)
        outputs = net(inputs)

        if iter_i == 0:
            d = outputs
        else:
            d = torch.cat((d, outputs), 0)

        iter_i += 1

In [ ]:
plt.scatter(test_df["Pawpularity"]/100, d.cpu().numpy().round(2), alpha=0.1)
plt.plot([0,1],[0,1],c='red',ls='-.')
